In [17]:
from sklearn.datasets import load_svmlight_file
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [66]:
X, y = load_svmlight_file('/Users/lianzitong/Desktop/MI/hw/hw11/digits/digit_train')
print X[0].shape
X = X.toarray()
print X.shape

(1, 771)
(500, 771)


In [67]:
# Add zero columns as margin of 28*28 squares
zeros = np.zeros((X.shape[0],28*28- X.shape[1]))
X = np.concatenate((X, zeros), axis = 1)
X_train = X
y_train = y

In [68]:
X= X.reshape((len(X), 28, 28))
print X.shape
print X_train.shape
# # Visualize first 12th 
# %matplotlib inline
# for i in range(12):
#     plt.figure(figsize=(1,1))
#     plt.imshow(X[i], cmap= 'gray')
#     plt.axis('off')
#     plt.title('label ' + str (int(y[i])))

(500, 28, 28)
(500, 784)


In [69]:
# Augmentation
# Rotation
aug = []
rows,cols = 28, 28
M = cv2.getRotationMatrix2D((cols/2,rows/2),-90,1)
for img in X:
    aug.append(cv2.warpAffine(img,M,(cols,rows)))
# Affine/Perspective Transformation
aug1 = []
pts1 = np.float32([[3,3],[26,26],[26,3]])
pts2 = np.float32([[2,2],[27,27],[27,2]])
M = cv2.getAffineTransform(pts1,pts2)
for img in X:
    aug1.append(cv2.warpAffine(img,M,(cols,rows)))

X_train = np.concatenate((X_train, np.asarray(aug1).reshape(X_train.shape), np.asarray(aug).reshape(X_train.shape)), axis = 0)
# Preprocessing
# sub mean, normalize
X_train1 = X_train- X_train.mean()
X_train1 = X_train1/255.0
# Shuffle?

In [53]:
# A simple svm
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
clf = SVC(C = 4, gamma = 0.03125)
# clf.fit(X, y) 
scores = cross_val_score(clf, X_train1, y_train, cv=10)
print scores    

[ 0.87272727  0.87037037  0.92307692  0.88        0.92        0.97959184
  0.91836735  0.875       0.89361702  0.7826087 ]


In [ ]:
# Best combi of C & gamma
# grid search cv
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
C_range = np.logspace(-6, 10, num= 9, base=2)
gamma_range = np.logspace(-5, 9, num= 8, base=2)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(X_train1, np.repeat(y_train, 3))
# Without image augmentation we reach cv_acc = 0.9
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))